# Hidden Inputs

> Generate hidden inputs for HTMX integration with keyboard navigation.

In [ ]:
#| default_exp htmx.inputs

In [ ]:
#| export
from __future__ import annotations
from fasthtml.common import Div, Hidden

from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone
from cjm_fasthtml_keyboard_navigation.core.manager import ZoneManager

from cjm_fasthtml_tailwind.utilities.layout import display_tw

## Zone Data Inputs

Generate hidden inputs for tracking focused item data attributes.

In [ ]:
#| export
def render_zone_hidden_inputs(
    zone: FocusZone  # the focus zone configuration
) -> list:           # list of Hidden input components
    """Render hidden inputs for a single zone's data attributes."""
    inputs = []
    for attr in zone.data_attributes:
        input_id = zone.get_hidden_input_id(attr)
        inputs.append(Hidden(id=input_id, name=attr, value=""))
    return inputs

In [ ]:
# Test zone inputs
from fasthtml.common import to_xml

zone = FocusZone(
    id="browser",
    item_selector="tr",
    data_attributes=("job-id", "plugin-name")
)

inputs = render_zone_hidden_inputs(zone)
assert len(inputs) == 2

html = to_xml(inputs[0])
assert 'id="browser-job-id"' in html
assert 'name="job-id"' in html

## Manager Hidden Inputs

Generate all hidden inputs for a ZoneManager.

In [ ]:
#| export
def render_hidden_inputs(
    manager: ZoneManager,            # the zone manager configuration
    include_state: bool = False,     # include state tracking inputs
    container_id: str = "kb-hidden-inputs"  # container element ID
) -> Div:                            # container with all hidden inputs
    """Render all hidden inputs for keyboard navigation."""
    inputs = []
    
    # Zone data attribute inputs
    for zone in manager.zones:
        inputs.extend(render_zone_hidden_inputs(zone))
    
    # State tracking inputs (if enabled)
    if include_state or manager.state_hidden_inputs:
        inputs.extend([
            Hidden(id="kb-active-zone", name="kb-active-zone", value=""),
            Hidden(id="kb-current-mode", name="kb-current-mode", value=""),
            Hidden(id="kb-focus-indices", name="kb-focus-indices", value=""),
        ])
    
    return Div(
        *inputs,
        id=container_id,
        cls=str(display_tw.hidden)
    )

In [ ]:
# Test manager inputs
from cjm_fasthtml_keyboard_navigation.core.manager import ZoneManager

zone1 = FocusZone(id="z1", data_attributes=("a",))
zone2 = FocusZone(id="z2", data_attributes=("b", "c"))

manager = ZoneManager(zones=(zone1, zone2))
container = render_hidden_inputs(manager)

html = to_xml(container)
assert 'id="kb-hidden-inputs"' in html
assert 'id="z1-a"' in html
assert 'id="z2-b"' in html
assert 'id="z2-c"' in html

In [ ]:
# Test with state inputs
container = render_hidden_inputs(manager, include_state=True)
html = to_xml(container)
assert 'id="kb-active-zone"' in html
assert 'id="kb-current-mode"' in html

## Include Selector Builder

Build `hx-include` selector for HTMX requests.

In [ ]:
#| export
def build_include_selector(
    zone: FocusZone,              # the zone to include inputs from
    include_state: bool = False   # include state inputs
) -> str:                         # CSS selector for hx-include
    """Build hx-include selector for zone's hidden inputs."""
    selectors = []
    
    for attr in zone.data_attributes:
        input_id = zone.get_hidden_input_id(attr)
        selectors.append(f"#{input_id}")
    
    if include_state:
        selectors.extend([
            "#kb-active-zone",
            "#kb-current-mode"
        ])
    
    return ", ".join(selectors)

In [ ]:
# Test include selector
zone = FocusZone(
    id="browser",
    data_attributes=("job-id", "plugin-name")
)

selector = build_include_selector(zone)
assert selector == "#browser-job-id, #browser-plugin-name"

selector_with_state = build_include_selector(zone, include_state=True)
assert "#kb-active-zone" in selector_with_state

In [ ]:
#| export
def build_all_zones_include_selector(
    manager: ZoneManager,         # the zone manager
    include_state: bool = False   # include state inputs
) -> str:                         # CSS selector for all zones
    """Build hx-include selector for all zones' hidden inputs."""
    selectors = []
    
    for zone in manager.zones:
        for attr in zone.data_attributes:
            input_id = zone.get_hidden_input_id(attr)
            selectors.append(f"#{input_id}")
    
    if include_state:
        selectors.extend([
            "#kb-active-zone",
            "#kb-current-mode",
            "#kb-focus-indices"
        ])
    
    return ", ".join(selectors)

In [ ]:
# Test all zones selector
selector = build_all_zones_include_selector(manager)
assert "#z1-a" in selector
assert "#z2-b" in selector
assert "#z2-c" in selector

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()